<a href="https://colab.research.google.com/github/Miguel2900/IC/blob/main/Proyecto_Final_IC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.datasets.mnist import load_data as dataNum
from tensorflow.keras.datasets.cifar10 import load_data as dataImg

import numpy as np
import cv2

In [17]:
(numTrnData,numTrnCategories),(numTstData,numTstCategories)=dataNum()
(imgTrnData,imgTrnCategories),(imgTstData,imgTstCategories)=dataImg()

In [18]:
numTrnData=numTrnData.reshape(numTrnData.shape[0],28,28,1)
numTstData=numTstData.reshape(numTstData.shape[0],28,28,1)
numFormat=numTrnData.shape[1:]

imgTrnData=np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in imgTrnData])
imgTstData=np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in imgTstData])
imgTrnData=imgTrnData.reshape(imgTrnData.shape[0],32,32,1)
imgTstData=imgTstData.reshape(imgTstData.shape[0],32,32,1)
imgFormat=imgTrnData.shape[1:]

numTstData.reshape(numTstData.shape[0],28,28,1)

numTrnData=numTrnData.astype('float32')/255.0
numTstData=numTstData.astype('float32')/255.0

imgTrnData=imgTrnData.astype('float32')/255.0
imgTstData=imgTstData.astype('float32')/255.0

In [21]:
datagenNum = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.25,
    height_shift_range = 0.25,
    zoom_range=[0.5,1.5],
)
datagenImg = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagenNum.fit(numTrnData)
datagenImg.fit(imgTrnData)

numTrnDataDAD = datagenNum.flow(numTrnData, numTrnCategories, batch_size=32)
imgTrnDataDAD = datagenImg.flow(imgTrnData, imgTrnCategories, batch_size=32)

In [22]:
numRegular=Sequential([
    Flatten(input_shape=(numFormat)), 
    Dense(units=50, activation='relu',kernel_initializer='he_uniform'),
    Dense(units=50, activation='relu',kernel_initializer='he_uniform'),
    Dense(10, activation='softmax')
])

numConv=Sequential([
    Conv2D(32,(3,3),activation='relu',kernel_initializer='he_uniform',input_shape=numFormat),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
])

numConvDAD=Sequential([
    Conv2D(32,(3,3),activation='relu',kernel_initializer='he_uniform',input_shape=numFormat),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
])

numRegular.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
numConv.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
numConvDAD.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [23]:
imgRegular=Sequential([
    Flatten(input_shape=(imgFormat)), 
    Dense(units=50, activation='relu',kernel_initializer='he_uniform'),
    Dense(units=50, activation='relu',kernel_initializer='he_uniform'),
    Dense(10, activation='softmax')
])

imgConv=Sequential([
    Conv2D(32,(3,3),activation='relu',kernel_initializer='he_uniform',input_shape=imgFormat),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
])

imgConvDAD=Sequential([
    Conv2D(32,(3,3),activation='relu',kernel_initializer='he_uniform',input_shape=imgFormat),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform'),
    MaxPooling2D(2,2),
    Conv2D(128,(3,3),activation='relu',kernel_initializer='he_uniform'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
])

imgRegular.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
imgConv.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
imgConvDAD.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epoch=60
batch=128

numRegular.fit(numTrnData,numTrnCategories,validation_split=0.25,epochs=epoch,batch_size=batch,verbose=1,use_multiprocessing=True)
numConv.fit(numTrnData,numTrnCategories,validation_split=0.25,epochs=epoch,batch_size=batch,verbose=1,use_multiprocessing=True)
numConvDAD.fit(numTrnDataDAD,epochs=epoch,batch_size=batch,verbose=1,use_multiprocessing=True)

imgRegular.fit(imgTrnData,imgTrnCategories,validation_split=0.25,epochs=epoch,batch_size=batch,verbose=1,use_multiprocessing=True)
imgConv.fit(imgTrnData,imgTrnCategories,validation_split=0.25,epochs=epoch,batch_size=batch,verbose=1,use_multiprocessing=True)
imgConvDAD.fit(imgTrnDataDAD,epochs=epoch,batch_size=batch,verbose=1,use_multiprocessing=True)

In [ ]:
numRegular.evaluate(numTstData,numTstCategories,verbose=1)
numConv.evaluate(numTstData,numTstCategories,verbose=1)
numConvDAD.evaluate(numTstData,numTstCategories,verbose=1)

imgRegular.evaluate(imgTstData,imgTstCategories,verbose=1)
imgConv.evaluate(imgTstData,imgTstCategories,verbose=1)
imgConvDAD.evaluate(imgTstData,imgTstCategories,verbose=1)

In [26]:
numRegular.save('numRegular.h5')
numConv.save('numConv.h5')
numConvDAD.save('numConvDAD.h5')

imgRegular.save('imgRegular.h5')
imgConv.save('imgConv.h5')
imgConvDAD.save('imgConvDAD.h5')

In [ ]:
!pip install tensorflowjs
!mkdir numRegular
!tensorflowjs_converter --input_format keras numRegular.h5 output

!mkdir numConv
!tensorflowjs_converter --input_format keras numConv.h5 numConv

!mkdir numConvDAD
!tensorflowjs_converter --input_format keras numConvDAD.h5 numConvDAD

!mkdir imgRegular
!tensorflowjs_converter --input_format keras imgRegular.h5 imgRegular

!mkdir imgConv
!tensorflowjs_converter --input_format keras imgConv.h5 imgConv

!mkdir imgConvDAD
!tensorflowjs_converter --input_format keras imgConvDAD.h5 imgConvDAD